<a href="https://colab.research.google.com/github/oyyarko/deeplearning_cnn/blob/master/fruit_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
os.chdir('/content/drive/My Drive/Deep Learning/Fruit Classifier')
#!curl 'https://storage.googleapis.com/kaggle-data-sets/5857/1107198/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1589009705&Signature=VRpcG5rNwNOn2BKIzU%2FtNRjREaVk0jo3j%2FVCixqHnkieBCcsqeKaxiNoIkuhdK3TyyQYoaTrs6tIjyznmFNoy1fVpME2kOehUFunZHc7CZsZEbx2LdjJMTz32Z6Txko6Bn7dA3zrxHGMtor7YUOyIYlNSlVxhXd6wWGG07O%2BdyGjXEUYiLYWEbYJBh5Drtpo0%2BmnSC3D1%2Fylj%2FcPMHH5GKOP3%2FFbuAvEQ2KOrPchzjV4n9fWJS2Se1Zk5M0mZDGe9Zuri%2Bt0wvKjmjk%2F%2BTvUY0k4vL3tLq39IffZcjsEZGzjy%2BKHY7xdB%2Fc2jjGjrf7t786REjbtrRjnGALayPR4Qw%3D%3D&response-content-disposition=attachment%3B+filename%3Dfruits.zip' -H 'User-Agent: Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:75.0) Gecko/20100101 Firefox/75.0' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' -H 'Accept-Language: en-US,en;q=0.5' --compressed -o data.zip

In [0]:
!unzip data.zip

In [0]:
#import libraries
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
import sys

In [0]:


#train_data_dir = sys.path.append('/fruit-360/Training')
#test_data_dir = sys.path.append('/fruit-360/Test')

train_data_dir = '/content/drive/My Drive/Deep Learning/Fruit Classifier/fruits-360/Training'
validation_data_dir = '/content/drive/My Drive/Deep Learning/Fruit Classifier/fruits-360/Test'

In [0]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

In [0]:
#create model
model = Sequential()
#CCMD
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(img_rows, img_cols, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

In [0]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [0]:
opt = RMSprop(learning_rate=0.001)
epochs = 10

checkpoint = ModelCheckpoint('fruit_classifier.h5', 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min')

earlystop = EarlyStopping(monitor='val_loss', 
                          min_delta=0, 
                          patience=3, 
                          verbose=1, 
                          mode='min', 
                          restore_best_weights=True)

reducelr = ReduceLROnPlateau(monitor='val_loss', 
                             factor=0.2, 
                             patience=3, 
                             verbose=1, 
                             mode='min', 
                             min_delta=0.0001)

In [0]:
model.compile(loss='categorical_crossentropy', 
              optimizer=opt,
              metrics=['accuracy'])

callbacks = [checkpoint, earlystop, reducelr]
nb_train_samples = 60486
nb_test_samples = 20618

history = model.fit_generator(train_generator,
                              steps_per_epoch = nb_train_samples//batch_size,
                              epochs=epochs,
                              verbose=1,
                              callbacks=callbacks,
                              validation_data=validation_generator,
                              validation_steps=nb_test_samples//batch_size,
                              shuffle=True)

In [0]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
Y_pred = model.predict_generator(validation_generator, nb_test_samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix:')
print(confusion_matrix(validation_generator.classes, y_pred))

In [0]:
class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}

print('Classification Report:')
target_names = list(class_labels.values())
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

In [0]:
import matplotlib.pyplot as plt
from keras.models import load_model

plt.figure(figsize=(20, 20))
cnf_matrix = confusion_matrix(validation_generator.classes, y_pred)
plt.imshow(cnf_matrix, interpolation='nearest')
plt.colorbar()
tick_marks = np.arange(len(target_names))
_ = plt.xticks(tick_marks, target_names, rotation=90)
_ = plt.yticks(tick_marks, target_names)

In [0]:
jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --NotebookApp.port_retries=0 --notebook-dir="" --no-browser --allow-root --NotebookApp.token='' --NotebookApp.disable_check_xsrf=True --port=8888

In [0]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
import re
from google.colab.patches import cv2_imshow

model = load_model('fruit_classifier.h5')

def draw_test(name, pred, im, true_label):
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 160, 0, 0, 500 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, "predited - "+ pred, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.putText(expanded_image, "true - "+ true_label, (20, 120) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,255,0), 2)
    #img = cv2.imread(name, expanded_image)
    cv2_imshow(expanded_image)


def getRandomImage(path, img_width, img_height):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    final_path = file_path + "/" + image_name
    return image.load_img(final_path, target_size = (img_width, img_height)), final_path, path_class

# dimensions of our images
img_width, img_height = 32, 32


files = []
predictions = []
true_labels = []
# predicting images
for i in range(0, 10):
    path = '/content/drive/My Drive/Deep Learning/Fruit Classifier/fruits-360/Test/'
    img, final_path, true_label = getRandomImage(path, img_width, img_height)
    files.append(final_path)
    true_labels.append(true_label)
    x = image.img_to_array(img)
    x = x * 1./255
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    classes = model.predict_classes(images, batch_size = 10)
    predictions.append(classes)
    
for i in range(0, len(files)):
    image = cv2.imread((files[i]))
    draw_test("Prediction", class_labels[predictions[i][0]], image, true_labels[i])
    cv2.waitKey(0)

cv2.destroyAllWindows()